In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
from keras.utils import np_utils

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/subject_lines_final.csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,source,subjects
0,0,Mary Fernandez,? a surprise gift for you! {unwrap}
1,1,Refinery29,10 bizarre money habits making Millennials richer
2,2,Manicube,Don’t Open This Email
3,3,Digital Marketer,Is this you?
4,4,Thrillist,What They Eat In Prison


In [ ]:
df_sub = df["subjects"] #Remove source, serial number columns
df_sub

0                    ? a surprise gift for you! {unwrap}
1      10 bizarre money habits making Millennials richer
2                                  Don’t Open This Email
3                                           Is this you?
4                                What They Eat In Prison
                             ...                        
381                   Happy Holidays from [Company Name]
382                       Invitation from [Company Name]
383                     Yes, this is a fundraising email
384                                       Best coat ever
385                                                  Hey
Name: subjects, Length: 386, dtype: object

In [ ]:
len(df_sub)

386

In [ ]:
def clean_text(txt):
  txt = "".join(v for v in txt if v not in string.punctuation).lower()
  txt = txt.encode("utf8").decode("ascii",'ignore')
  return txt 


In [ ]:
corpus = [clean_text(x) for x in df_sub]
corpus[:10]

[' a surprise gift for you unwrap',
 '10 bizarre money habits making millennials richer',
 'dont open this email',
 'is this you',
 'what they eat in prison',
 '9 disgusting facts about thanksgiving',
 'hologram shorts',
 'check out my new man cave pics',
 'is this the hottest career in marketing',
 'start using this new facebook ad type']

In [ ]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  ## tokenization
  tokenizer.fit_on_texts(corpus) #Updates internal vocabulary based on a list of texts and word frequencies. High frequency=>lower index(appears in the beginning of the list). 
  #Eg: "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2... 
  total_words = len(tokenizer.word_index) + 1
    
  ## convert data to sequence of tokens 
  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] #Transforms each 'line' in 'corpus' to a sequence of integers. It takes each wordand replaces it with its corresponding integer value from the word_index dictionary.
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]#For each word, an n-gram sequence is made and input sequences are updated. 
      input_sequences.append(n_gram_sequence)
  return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[4, 96],
 [4, 96, 61],
 [4, 96, 61, 10],
 [4, 96, 61, 10, 2],
 [4, 96, 61, 10, 2, 230],
 [49, 150],
 [49, 150, 62],
 [49, 150, 62, 97],
 [49, 150, 62, 97, 151],
 [49, 150, 62, 97, 151, 152]]

In [ ]:
def generate_padded_sequences(input_sequences):
  #The maximum length of the sentence is extracted and then the rest of the sentences are pre-padded as per the longest sentence.
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

  #Extract the last word of sequence and convert it to categorical from numerical.    
  predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
  label = ku.np_utils.to_categorical(label, num_classes=total_words)
  return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()
  # Add Input Embedding Layer
  model.add(Embedding(total_words,6,input_length=input_len))
    
  # Add Hidden Layer 1 - LSTM Layer
  model.add(LSTM(50))
  model.add(Dropout(0.1))
    
  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')
    
  return model

model = create_model(max_sequence_len, total_words) 
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 15, 6)             5250      
                                                                 
 lstm_3 (LSTM)               (None, 50)                11400     
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 875)               44625     
                                                                 
Total params: 61,275
Trainable params: 61,275
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='post')
    #predicted = model.predict_classes(token_list, verbose=0)
    predicted = model.predict(token_list, verbose=0)

    output_word = ""
    for word,index in tokenizer.word_index.items():
      #if index == predicted:
      #if index == predicted.any():
      if index == predicted.all():
        output_word = word
        break
    seed_text += " "+output_word
  return seed_text.title()

In [ ]:
print (generate_text("tonight",5, model, max_sequence_len))
print (generate_text("season pass",3, model, max_sequence_len))
print (generate_text("Enjoy NBA",5, model, max_sequence_len))
print (generate_text("get more",6, model, max_sequence_len))

Tonight Your Your Your Your Your
Season Pass Your Your Your
Enjoy Nba Your Your Your Your Your
Get More Your Your Your Your Your Your
